Linear regression using

Boston Housing Data

dataset docs: https://www.kaggle.com/code/prasadperera/the-boston-housing-dataset/data

In [21]:
# import packages
import pandas as pd
import numpy as np
from pathlib import Path
import math

from sklearn.datasets import fetch_california_housing

import math, copy


In [22]:
housing = fetch_california_housing()

housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [102]:
# np.random.seed(1)
np.random.randint(10)

0

In [96]:
# helpers - data clean

def load_data():
    housing  = fetch_california_housing()
    
    df = pd.DataFrame(data= np.c_[housing['data'], housing['target']],
                     columns= housing['feature_names'] + ['target'])
    return df
    

def train_test_split(df):
    n = len(df)
    
    # train test split (2/3 train, 1/3 test)
    n_train = round(2/3*n)

    train_df = df[:n_train]
    test_df = df[n_train:]
    
    return train_df, test_df

In [24]:
# helpers - single variable linear regression
# my version that was coded from scratch

def df_to_input(df):
    m = len(df)
    X = df['AveBedrms'].values.reshape(1, m)
    Y = df['target'].values.reshape(1, m)
    
    return X, Y


def initial_rand(X):
    
    np.random.seed(1)
    
    m = X.shape[0]
    n = X.shape[1]
    
    w = np.random.randn(n).reshape(n,1) * 0.01
    b = np.random.randint(0,100) * 0.01 
    
    return w, b
    
    
def single_var_backprop(X, Y, w_in, b_in, Y_hat, learning_rate = 0.005, num_iters = 10000):
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = copy.deepcopy(b_in)  
    
    for i in range (num_iters):
        db, dw = calc_grads(X, Y, Y_hat)
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append(compute_cost(X, Y_hat, Y))

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}   ")
        
    return w, b, J_history



In [25]:
# run

df = load_data()

train_df, test_df = train_test_split(df) 


In [26]:
train_df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
13755,2.8125,33.0,5.099515,1.135922,1997.0,4.847087,34.06,-117.16,1.172
13756,1.8152,17.0,4.223660,1.024030,1412.0,2.609982,34.06,-117.16,0.943
13757,7.2972,26.0,7.471344,1.038538,2909.0,2.874506,34.01,-117.14,2.696
13758,6.5648,32.0,7.355844,1.020779,1033.0,2.683117,34.03,-117.15,2.372


In [27]:
# show data
# m = number of training examples
m = train_df.values.shape[0]
# n = number of features
n = len(train_df.drop(columns='target').columns)

m, n

(13760, 8)

In [28]:
# X should be of the dimensions m, n

X = train_df.drop(columns='target').values.reshape(m,n)
Y = train_df['target'].values.reshape(m,1)

X.shape, Y.shape



((13760, 8), (13760, 1))

In [152]:
# example from machine learning specialisation


X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])

X_train.shape

def predict_single_loop(x, w, b): 
    """
    single predict using linear regression
    
    Args:
      x (ndarray): Shape (n,) example with multiple features
      w (ndarray): Shape (n,) model parameters    
      b (scalar):  model parameter     
      
    Returns:
      p (scalar):  prediction
    """
    n = x.shape[0]
    p = 0
    for i in range(n):
        p_i = x[i] * w[i]  
        p = p + p_i         
    p = p + b                
    return p

b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])
print(f"w_init shape: {w_init.shape}, b_init type: {type(b_init)}")

# get a row from our training data
x_vec = X_train[0,:]
print(f"x_vec shape {x_vec.shape}, x_vec value: {x_vec}")


def forward_prop(X, w, b):
    n = X.shape[0]
    # reshape step important for later functions
    Y_hat = np.dot(X_train, w_init) + b_init
    return Y_hat

# ml spec function  prediction
f_wb = predict_single_loop(x_vec, w_init, b_init)
print(f"f_wb shape {f_wb.shape}, prediction: {f_wb}")

# shows that prediction value was the same for for loop as for cevotrised implementation ( in my code)
# this shows my code is right currently
g_wb = forward_prop(X_train, w_init, b_init)[0]
print(f"g_wb shape {g_wb.shape}, prediction: {g_wb}")

assert g_wb == f_wb

if g_wb == f_wb:
    print('all tests passed')



w_init shape: (4,), b_init type: <class 'float'>
x_vec shape (4,), x_vec value: [2104    5    1   45]
f_wb shape (), prediction: 459.9999976194083
g_wb shape (), prediction: 459.9999976194083
all tests passed


In [146]:
(np.dot(X_train, w_init) + b_init)[0]

459.9999976194083

In [139]:
# compute cost code comparisons

X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])
b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])


def ml_compute_cost(X, y, w, b): 
    """
    compute cost
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """
    m = X.shape[0]
    cost = 0.0
    for i in range(m):                                
        f_wb_i = np.dot(X[i], w) + b           #(n,)(n,) = scalar (see np.dot)
        cost = cost + (f_wb_i - y[i])**2       #scalar
    cost = cost / (2 * m)                      #scalar    
    return cost

# Compute and display cost using our pre-chosen optimal parameters. 
ml_cost = ml_compute_cost(X_train, y_train, w_init, b_init)
print(f'Cost at optimal w : {ml_cost}', type(ml_cost))


# correct, vectorised implementation of the cots function
m = X_train.shape[0]
Y_hat = np.dot(X_train, w_init) + b_init
cost = np.sum((Y_hat - y_train)**2 ) / (2*m)

print(f'Cost at optimal w : {cost}', type(cost))

# function version of vectorised implementation
def calculate_cost(X, Y_train, w, b):
    m = X_train.shape[0]
    Y_hat = forward_prop(X_train, w, b)
    cost = np.sum((Y_hat - y_train)**2 ) / (2*m)
    return cost

f_cost = calculate_cost(X_train, y_train, w_init, b_init)
print(f'Cost at optimal w : {f_cost}', type(f_cost))

assert f_cost == ml_cost

if f_cost == ml_cost:
    print('all tests passed')

Cost at optimal w : 1.5578904045996674e-12 <class 'numpy.float64'>
Cost at optimal w : 1.5578904045996674e-12 <class 'numpy.float64'>
Cost at optimal w : 1.5578904045996674e-12 <class 'numpy.float64'>
all tests passed


In [68]:
def ml_compute_gradient(X, y, w, b): 
    """
    Computes the gradient for linear regression 
    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
      dj_db (scalar):       The gradient of the cost w.r.t. the parameter b. 
    """
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.
    
    # for each training and each feature example calculate the derivitives then....
    # take the average accross number of training examples
    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]   
        # for each feature
        for j in range(n):
            # dw = 0 + y_hat - y * X
            dj_dw[j] = dj_dw[j] + err * X[i, j]   
        dj_db = dj_db + err
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_db, dj_dw

# resetting variables
X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])
b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])

m,n = X_train.shape           #(number of examples, number of features)


# ml_spec compute and display gradient 
tmp_dj_db, tmp_dj_dw = ml_compute_gradient(X_train, y_train, w_init, b_init)
print(f'dj_db at initial w,b: {tmp_dj_db}')
print(f'dj_dw at initial w,b: \n {tmp_dj_dw}')

dw = 0 
db = 0

Y_hat = forward_prop(X_train, w_init, b_init)

db = np.mean(Y_hat - y_train)
dw = np.sum(((Y_hat - y_train) * X_train.T), axis=1) / m
print(f'dw at initial w,b: \n {db}')
print(f'db at initial w,b: {dw}')

def calculate_grads(X_train, Y_train, w, b):
    m, n = X_train.shape
    Y_hat = forward_prop(X_train, w, b)
    db = np.mean(Y_hat - Y_train)
    dw = np.sum(((Y_hat - Y_train) * X_train.T), axis=1) / m
    
    return dw, db

f_dw, f_db = calculate_grads(X_train, y_train, w_init, b_init)
print(f'f_dw at initial w,b: \n {f_db}')
print(f'f_db at initial w,b: {f_dw}')

assert f_dw[0] == tmp_dj_dw[0]
assert f_db == tmp_dj_db

if  f_dw[0] == tmp_dj_dw[0]:
    print('all tests passed')

dj_db at initial w,b: -1.6739251122999121e-06
dj_dw at initial w,b: 
 [-2.72623574e-03 -6.27197255e-06 -2.21745574e-06 -6.92403377e-05]
dw at initial w,b: 
 -1.6739251122999121e-06
db at initial w,b: [-2.72623574e-03 -6.27197255e-06 -2.21745574e-06 -6.92403377e-05]
f_dw at initial w,b: 
 -1.6739251122999121e-06
f_db at initial w,b: [-2.72623574e-03 -6.27197255e-06 -2.21745574e-06 -6.92403377e-05]
all tests passed


grad returning incorrect still on below function despite above tests passing

In [133]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    """
    Performs batch gradient descent to learn theta. Updates theta by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X (ndarray (m,n))   : Data, m examples with n features
      y (ndarray (m,))    : target values
      w_in (ndarray (n,)) : initial model parameters  
      b_in (scalar)       : initial model parameter
      cost_function       : function to compute cost
      gradient_function   : function to compute the gradient
      alpha (float)       : Learning rate
      num_iters (int)     : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,)) : Updated values of parameters 
      b (scalar)       : Updated value of parameter 
      """
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db,dj_dw = gradient_function(X, y, w, b)   ##None
        
#         print(f'dj_db on iter {i}:',dj_db)
#         print(f'dj_dw on iter {i}:',dj_dw)
            
        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               ##None
        b = b - alpha * dj_db               ##None
      
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( cost_function(X, y, w, b))

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}   ")
        
#         if i ==2:
#             break
        
    return w, b, J_history #return final w,b and J history for graphing

In [134]:
# initialize parameters
initial_w = np.zeros_like(w_init)
initial_b = 0.

# some gradient descent settings
iterations = 1000
alpha = 5.0e-7

# run gradient descent 
w_final, b_final, J_hist = gradient_descent(X_train, y_train, initial_w, initial_b,
                                                    ml_compute_cost, ml_compute_gradient, 
                                                    alpha, iterations)
print(f"b,w found by gradient descent: {b_final:0.2f},{w_final} ")
m,_ = X_train.shape
for i in range(m):
    print(f"prediction: {np.dot(X_train[i], w_final) + b_final:0.2f}, target value: {y_train[i]}")

[0. 0. 0. 0.]
Iteration    0: Cost  2529.46   
Iteration  100: Cost   695.99   
Iteration  200: Cost   694.92   
Iteration  300: Cost   693.86   
Iteration  400: Cost   692.81   
Iteration  500: Cost   691.77   
Iteration  600: Cost   690.73   
Iteration  700: Cost   689.71   
Iteration  800: Cost   688.70   
Iteration  900: Cost   687.69   
b,w found by gradient descent: -0.00,[ 0.20396569  0.00374919 -0.0112487  -0.0658614 ] 
prediction: 426.19, target value: 460
prediction: 286.17, target value: 232
prediction: 171.47, target value: 178


In [135]:
def gradient_descent(X, y, w_in, b_in, compute_cost, compute_grads, learning_rate, num_iters):

    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in

    for i in range(num_iters):

        # calculate gradient & updte params
        dw, db = calculate_grads(X, y, w, b)

#         print(f'db on iter {i}:',db)
#         print(f'dw on iter {i}:',dw)
        
        w = w - learning_rate * dw
        b = b - learning_rate * db

        if i < 100000: 
            J_history.append(compute_cost(X, y, w, b))

        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}   ")
        
#         if i ==2:
#             break    
        
        return w, b, J_history #return final w,b and J history for graphing

In [136]:
# initialize parameters
initial_w = np.zeros_like(w_init)
initial_b = 0.

# some gradient descent settings
iterations = 1000
alpha = 5.0e-7
# run gradient descent 
w_final, b_final, J_hist = gradient_descent(X_train, y_train, initial_w, initial_b,
                                                    calculate_cost, calculate_grads, 
                                                    alpha, iterations)
print(f"b,w found by gradient descent: {b_final:0.2f},{w_final} ")
m,_ = X_train.shape
for i in range(m):
    print(f"prediction: {np.dot(X_train[i], w_final) + b_final:0.2f}, target value: {y_train[i]}")

Iteration    0: Cost     0.00   
b,w found by gradient descent: 0.00,[1.36311787e-09 3.13598628e-12 1.10872787e-12 3.46201689e-11] 
prediction: 0.00, target value: 460
prediction: 0.00, target value: 232
prediction: 0.00, target value: 178


In [ ]:
X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])
b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])

w, b = initial_rand(X_train)
# print('w',w,'b',b)

Y_hat = forward_prop(X_train, w, b)[0]
# print('yhat',y_hat.shape)
Y_hat

w, b, J_history = single_var_backprop(X_train, y_train, w_init, b_init, Y_hat)
# print(w, b)

In [ ]:
X.shape, Y.shape